# YOLO - Training
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jul 2021<br>
**Descripción:** Notebook de entrenamiento.<br>

## Instalación de OpenCV

``` bash
    sudo apt install python3-opencv
    sudo apt-get install libopencv-dev
```

## Instalar zip

``` bash
    sudo apt install zip unzip
``` 

## Preparación del entorno

* Crear entorno virtual (solo la primera vez) dentro de la carpeta `/training`.

``` bash
    python3 -m venv yolo_training --system-site-packages # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source yolo_training/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r env # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

* Agregar kernel a JupyterLab (**con el entorno virtual activado**).
    
``` bash
    python -m ipykernel install --user --name=yolo_training 
```

## Clonación y construcción de Darknet (por única vez)

In [ ]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

## Definir funciones auxiliares

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

## Entrenamiento personalizado
### Descargando conjunto de datos por URL

**OpenImage**

obj_data URL: https://drive.google.com/file/d/1kUy8UCKNs3gwZz4Gw67FhHgKn90a-Muo/view?usp=sharing

test_data URL: https://drive.google.com/file/d/1SqbKygmdH7VWahN9Zym2-a9RFOgVx3of/view?usp=sharing

readme URL: https://drive.google.com/file/d/1QmE8DLbBoy7e1nTM3YhGBVMbNfVP14m_/view?usp=sharing

In [ ]:
%cd /content

In [ ]:
# obj_data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kUy8UCKNs3gwZz4Gw67FhHgKn90a-Muo' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1kUy8UCKNs3gwZz4Gw67FhHgKn90a-Muo" -O obj.zip && rm -rf /tmp/cookies.txt
# test_data
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SqbKygmdH7VWahN9Zym2-a9RFOgVx3of' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1SqbKygmdH7VWahN9Zym2-a9RFOgVx3of" -O test.zip && rm -rf /tmp/cookies.txt

In [ ]:
# descomprima los conjuntos de datos y su contenido para que ahora estén en la carpeta /darknet/data/
!unzip obj.zip -d darknet/data/
!unzip test.zip -d darknet/data/

# eliminando carpetas
!rm -rf obj.zip
!rm -rf test.zip

### Copiando archivos de configuración


In [ ]:
cp data_training/obj.data darknet/data/obj.data
cp data_training/obj.names darknet/data/obj.names
cp data_training/yolov4-obj.cfg darknet/cfg/yolov4-obj.cfg

### Descargando archivos de configuración (Solo en Colab)


In [ ]:
# descargando obj.data, obj.names y yolov4-obj.cfg (comprimidos)
!wget -O darknet/data/obj.data https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training.zip?raw=true

In [ ]:
!unzip data_training.zip -d data_training

In [ ]:
cp data_training/obj.data darknet/data/obj.data
cp data_training/obj.names darknet/data/obj.names
cp data_training/yolov4-obj.cfg darknet/cfg/yolov4-obj.cfg

In [ ]:
!rm -rf data_training.zip

### Generando train.txt y test.txt

In [ ]:
%cd /content/darknet/

In [ ]:
import os

In [ ]:
# Generando test.txt

image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [ ]:
# Generando train.txt

image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

### Descargue pesos pre-entrenados para las capas convolucionales.

In [ ]:
!wget -O pre_training/ https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### Entrena tu detector personalizado!

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# muestre chart.png de cómo le fue al detector de objetos personalizado con la capacitación
imShow('chart.png')

 ### Iniciar el entrenamiento desde donde se guardó por última vez

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/ml_uali/training/yolo_v1/checkpoints/yolov4-obj_last.weights -dont_show -map

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/ml_uali/training/yolo_v1/checkpoints/yolov4-obj_final.weights